# Initial Web Scrape for Movie Scripts

## Importing Necessary Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [3]:
import time
import requests
from selenium import webdriver
import chromedriver_binary
# from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re


## Establishing URL

In [199]:
genre_site = "https://imsdb.com/genre/Action"
get_all = "https://imsdb.com/all-scripts.html"
url = "https://imsdb.com/scripts/Avengers,-The-(2012).html"
nurse_betty = "https://imsdb.com/Movie%20Scripts/Nurse%20Betty%20Script.html"

In [200]:
nurse = requests.get(nurse_betty)
nurse_soup = BeautifulSoup(nurse.content, 'html.parser')

In [202]:
nurse_soup.find_all('table', align="center")[0].find_all('a')

[<a href="/writer.php?w=John C. Richards" title="Scripts by John C. Richards">John C. Richards</a>,
 <a href="/writer.php?w=James Flamberg" title="Scripts by James Flamberg">James Flamberg</a>,
 <a href="/genre/Comedy" title="Comedy Scripts">Comedy</a>,
 <a href="/genre/Thriller" title="Thriller Scripts">Thriller</a>,
 <a href="/scripts/Nurse-Betty.html">Read "Nurse Betty" Script</a>]

In [198]:
r = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')
soup.prettify()

'<html>\n <head>\n  <meta content="width=device-width, initial-scale=1" name="viewport"/>\n  <meta content="true" name="HandheldFriendly"/>\n  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>\n  <meta content="EN" http-equiv="Content-Language"/>\n  <meta content="Document" name="objecttype"/>\n  <meta content="INDEX, FOLLOW" name="ROBOTS"/>\n  <meta content="Movie scripts, Film scripts" name="Subject"/>\n  <meta content="General" name="rating"/>\n  <meta content="Global" name="distribution"/>\n  <meta content="2 days" name="revisit-after"/>\n  <link href="/style.css" rel="stylesheet" type="text/css"/>\n  <script type="text/javascript">\n   var _gaq = _gaq || [];\r\n  _gaq.push([\'_setAccount\', \'UA-3785444-3\']);\r\n  _gaq.push([\'_trackPageview\']);\r\n\r\n  (function() {\r\n    var ga = document.createElement(\'script\'); ga.type = \'text/javascript\'; ga.async = true;\r\n    ga.src = (\'https:\' == document.location.protocol ? \'https://ssl\' : \'http://www\') +

## Making Soup

In [128]:
# Previewing one script within www.imsdb.com
soup.find('pre')

<pre>




<b>                                    THE AVENGERS
</b>



                                     Written by

                                    Joss Whedon                        

                         

                         

                         
          "And there came a day, a day unlike any other, when Earth's
          mightiest heroes and heroines found themselves united against a
          common threat. On that day, the Avengers were born--to fight the
          foes no single superhero could withstand! Through the years,
          their roster has prospered, changing many times, but their glory
          has never been denied! Heed the call, then--for now, the Avengers
          Assemble!"

          BURNING BLUE FLAMES. A smoky cube shape emerges - THE TESSERACT.
          Filling the screen with BLACKNESS.

<b>                          CUT TO:
</b>
<b>          EXT. THRONE ROOM, SPACE ­ NIGHT
</b>
          Kneeling behind a THRONE, a CLOTHED, ARMOR

In [103]:
#Removing unneccessary characters and leaving only text
results = soup.find('pre').text
results2 = re.sub("[^a-zA-Z'. ]+", '', results)
results2 = results2.strip('[     ]')
results2 = [re.sub(r'  ', " ", results2) for i in range(10)]
type(results2)

list

In [102]:
#Previewing stage directions and character names listed in tag b
resb = soup.find_all('b')

lst=[]
    
for i in range(len(resb)):
    k = resb[i].text
    k = k.strip('[     ]')
    k = re.sub("[^a-zA-Z' ]+", '', k)
    lst.append(k)
    
stage = lst[:-4]
print(len(stage))
type(stage)

1336


list

In [119]:
res = soup.find_all('pre')

lst2=[]
    
for i in range(len(res)):
    k = res[i].text
#     k = k.strip('[     ]')
#     k = re.sub("[^a-zA-Z' ]+", '', k)
    lst2.append(k)

print(len(lst2[0]))
type(lst2[0])

202763


str

In [87]:
for i in range(len(resb[:-3])):
    k = resb[i].text
    
for i in range(len(resb[:-3])):
    something = resb[i].text
#     print(something)



In [88]:
# for t in resb:
#     if not resb.find('b'):
#         continue
#     print(t.text)

another = [i.text for i in soup.find('pre') if not soup.find_all('b')]
print(len(soup.find('pre')))
print(len(soup.find_all('b')))

print(another)

2677
1340
[]


In [74]:
ch = soup.find('pre')

In [75]:
i = ch.find_all('b')

## Trying to get all movies

In [129]:
allmov = requests.get(get_all)
soup3 = BeautifulSoup(allmov.content, 'html.parser')

In [131]:
all_movies = soup3.find_all('td',  valign="top")[2].find_all('a')

In [132]:
movies = [(movie_info.string, \
              movie_info["href"], \
              re.split("[,.]",movie_info.string)[0].replace(' ', '_'))
              for movie_info in all_movies]

In [220]:
len(movies)

1210

In [142]:
def spot_check(movies):
    for movie in movies:
        if movie[1][:15] != "/Movie Scripts/":
            return "uh oh"
        return "All good!"

In [162]:
def handle_movie (movie, browser):

    # Unpack tuple
    title, link_to_movie_page, movie_title = movie

    # Interrogate the page with all the movie information (ratings, writer,
    # genre, link to script)
    full_html_link = u'http://www.imsdb.com' + link_to_movie_page
    response_script = requests.get(full_html_link)
    soup = BeautifulSoup(response_script.text, 'html.parser')

    # Get all relevant information (genre, writer, script) from page
    list_links = soup.find_all('table', "script-details")[0].find_all('a')
    print(len(list_links))
    genre = []
    writer = []
    script = ''
    for link in list_links:
        href = link['href']
        if href[0:7]== "/writer":
            writer.append(link.get_text())
        if href[0:7]== "/genre/":
            genre.append(link.get_text())
        if href[0:9]== "/scripts/":
            script = href
    

In [206]:
list_links = soup.find_all('table', align="center")[0].find_all('a')
list_links

[<a href="/writer.php?w=Joss Whedon" title="Scripts by Joss Whedon">Joss Whedon</a>,
 <a href="/genre/Action" title="Action Scripts">Action</a>,
 <a href="/genre/Adventure" title="Adventure Scripts">Adventure</a>,
 <a href="/genre/Sci-Fi" title="Sci-Fi Scripts">Sci-Fi</a>,
 <a href="/Movie Scripts/Avengers, The (2012) Script.html#comments" title="Avengers, The (2012) comments">User Comments</a>]

In [184]:
for link in list_links:
    print(link)

<a href="/writer.php?w=Joss Whedon" title="Scripts by Joss Whedon">Joss Whedon</a>
<a href="/genre/Action" title="Action Scripts">Action</a>
<a href="/genre/Adventure" title="Adventure Scripts">Adventure</a>
<a href="/genre/Sci-Fi" title="Sci-Fi Scripts">Sci-Fi</a>
<a href="/Movie Scripts/Avengers, The (2012) Script.html#comments" title="Avengers, The (2012) comments">User Comments</a>


In [189]:
genre = []
writer = []
script = ''
for link in list_links:
        href = link['href']
        if href[0:7]== "/writer":
            writer.append(link.get_text())
        if href[0:7]== "/genre/":
            genre.append(link.get_text())
        if href[0:9]== "/scripts/":
            script = href
            

print(genre)

print(writer)
print(script)

['Action', 'Adventure', 'Sci-Fi']
['Joss Whedon']



## Other Attempts

In [77]:
for i in ch.find_all('b'):
    i.decompose

In [78]:
ch

<pre>




<b>                                    THE AVENGERS
</b>



                                     Written by

                                    Joss Whedon                        

                         

                         

                         
          "And there came a day, a day unlike any other, when Earth's
          mightiest heroes and heroines found themselves united against a
          common threat. On that day, the Avengers were born--to fight the
          foes no single superhero could withstand! Through the years,
          their roster has prospered, changing many times, but their glory
          has never been denied! Heed the call, then--for now, the Avengers
          Assemble!"

          BURNING BLUE FLAMES. A smoky cube shape emerges - THE TESSERACT.
          Filling the screen with BLACKNESS.

<b>                          CUT TO:
</b>
<b>          EXT. THRONE ROOM, SPACE ­ NIGHT
</b>
          Kneeling behind a THRONE, a CLOTHED, ARMOR

In [79]:
# repl = re.sub("[^a-zA-Z' ]+", '', lst)
# print(repl)

In [101]:
regex = re.compile(r'[\n\r\xad\\]')
e = regex.sub(" ", results)
t = e.strip()

type(results)

str

In [98]:
regex = re.compile(r'  ')

for i in range(10):
    t = regex.sub(" ", t)
type(t)

str

In [97]:
replaced = re.sub("[^a-zA-Z' ]+", '', t)
print(len(replaced))
replaced

129399


'A'

In [ ]:
# first_result.find('a')['href'] # for html

In [ ]:
r2 = soup.find("b", class_=False, id=False)
# r2.find_next_sibling(text=True)


In [ ]:
# soup2.title.contents

In [ ]:
tri = soup2.find_all("pre")
# tri[1]